# **Starting Work**

In [ ]:
from os import chdir, environ

In [ ]:
global logging_init_flag
logging_init_flag = False

def init_logging(OUTPUT):
    global logging_init_flag
    if logging_init_flag:
        return
    logging_init_flag = True

    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    formatter = logging.Formatter(
        '%(asctime)s - %(levelname)s:\t%(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')

    fh = logging.FileHandler(f"{OUTPUT}/training.log")
    fh.setLevel(logging.INFO)
    fh.setFormatter(formatter)

    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    ch.setFormatter(formatter)

    logger.addHandler(ch)
    logger.addHandler(fh)



In [ ]:
!git pull

In [ ]:
!ls -lt --time-style='+%y-%m-%d %H:%M:%S'
!dir

# **Experiment Init**

In [ ]:
import json
import h5py
import torch
import os
from encoder import *
# from encoder import MLP
# from encoder_resmlp import *
from res_encoder import ResEncoder
from dyn_encoder import DynEncoder, SampleEncoder
from build_tree import get_directions, init_directions
import logging
from dataset import *

prefix = "_affine_xinf"
no_prealign = False 
rotate_only = False
test_as_valid = False #False
augment = 1
transform = affine_transform 
use_symmetry_loss = False
dim = 4096 # 4096
lowrk = 999999999
dim_layer0 = 32 # 32
dim_repeat_cut = 5 # 5
inner_dim = 64
train_augment = False
permute_augment = False #True
model_size = 2 ** 11
num_layers = 1 
num_res_layers = 1
carry_dim = dim_layer0 * 8
upload_remove_equal = False # False
use_small_point2embed = False
model_name = 'ResEncoder' 
pca_augment = False # True
augment_fn = lambda pts : (pts, None) #augment_generator(pts, dropout=0.875, shift=True, jitter=True, scale=True)
disable_alignment = False
extra_sample_coef = 1/9
coo_dim = 4
catmlp = False
use_norm = False
complete_dataset = False
augment_fn = lambda pts : (pts, None) #augment_generator(pts, dropout=0.875, shift=True, jitter=True, scale=True)


# prefix = "_affine_pdnet_nopa_x4"
# no_prealign = True 
# rotate_only = False
# test_as_valid = False
# augment = 4
# transform = affine_transform 
# use_symmetry_loss = False
# dim = 1024 # 4096
# lowrk = 999999999
# dim_layer0 = 8 # 32
# dim_repeat_cut = 5 # 5
# train_augment = False
# permute_augment = False #True
# model_size = 2 ** 11
# num_layers = 1
# num_res_layers = 1
# carry_dim = dim_layer0 * 8
# upload_remove_equal = False # False
# use_small_point2embed = False
# model_name = 'Encoder' 
# pca_augment = False # True
# augment_fn = lambda x : x
# disable_alignment = False
# catmlp = True
# use_norm = False
# complete_dataset = False
# augment_fn = lambda pts : (pts, None) #augment_generator(pts, dropout=0.875, shift=True, jitter=True, scale=True)

# prefix = "_orig_cpp_tav_complete_norm_xinf"
# no_prealign = True 
# rotate_only = False
# use_norm = True
# complete_dataset = True
# test_as_valid = True
# augment = 1
# transform = lambda x : x
# use_symmetry_loss = True
# dim = 4096 # 4096
# lowrk = 999999999
# dim_layer0 = 32 # 32
# dim_repeat_cut = 5 # 5
# train_augment = False
# permute_augment = False #True
# model_size = 2 ** 11
# num_layers = 1
# num_res_layers = 1
# carry_dim = dim_layer0 * 8
# upload_remove_equal = False # False
# use_small_point2embed = False
# model_name = 'Encoder' 
# pca_augment = False # True
# # augment_fn = lambda pts : augment_generator(pts, fetch_perm=use_norm, dropout=0.5, shift=True, jitter=True, scale=True, agg_coef=0.5) # dropout=0.875
# augment_fn = lambda pts : augment_generator(pts, fetch_perm=use_norm) #, fetch_perm=use_norm, shift=True, scale=True)
# disable_alignment = False
# catmlp = False

# affine new
# prefix = "_affine_dyn_xinf"
# no_prealign = False 
# rotate_only = False
# test_as_valid = False
# augment = 1
# transform = affine_transform 
# use_symmetry_loss = False
# dim = 1024 # 4096
# lowrk = 999999999
# dim_layer0 = 32 # 32
# dim_repeat_cut = 0 # 5
# inner_dim = 64
# train_augment = False
# permute_augment = False #True
# model_size = 2 ** 11
# num_layers = 1 
# num_res_layers = 1
# carry_dim = dim_layer0 * 8
# upload_remove_equal = False # False
# use_small_point2embed = False
# model_name = 'SampleEncoder' 
# pca_augment = False # True
# augment_fn = lambda pts : (pts, None) #augment_generator(pts, dropout=0.875, shift=True, jitter=True, scale=True)
# disable_alignment = False
# extra_sample_coef = 1/9
# coo_dim = 4


# orig
# prefix = "_orig_cpp_tav_xinf"
# no_prealign = True 
# rotate_only = False
# test_as_valid = True
# augment = 1
# transform = lambda x : x
# use_symmetry_loss = False
# dim = 4096 # 4096
# lowrk = 999999999
# dim_layer0 = 32 # 32
# dim_repeat_cut = 5 # 5
# train_augment = False
# permute_augment = False #True
# model_size = 2 ** 11
# num_layers = 1
# num_res_layers = 1
# carry_dim = dim_layer0 * 8
# upload_remove_equal = False # False
# use_small_point2embed = False
# model_name = 'Encoder' 
# pca_augment = False # True
# augment_fn = lambda pts : augment_generator(pts, dropout=0.875, shift=True, jitter=True, scale=True, rotate_yaxis=False)
# disable_alignment = False


OUTPUT = 'scratch'

sample_layers = 50 # 2
channel = 1


sample_child_first = False # True in l7s1
num_classes = 40
DATASET = './datasets/ModelNet' if complete_dataset else './datasets/ModelNet40'
chaos_limit = 0

init_logging(OUTPUT)
logging.info(f"prefix = {prefix}")
_ = init_directions(chaos_limit, calc_dmap=False)


In [ ]:
torch.manual_seed(674433238)
has_align_feature = False
if model_name == 'Encoder':
    model = Encoder(model_size, sample_layers, dim, OUTPUT, catmlp=catmlp, extra_dim=3 if use_norm else 0, channel=channel, sample_child_first=sample_child_first, dim_layer0=dim_layer0, dim_repeat_cut=dim_repeat_cut, use_symmetry_loss=use_symmetry_loss).cuda()
elif model_name == 'ResEncoder':
    has_align_feature = True
    model = ResEncoder(num_res_layers, carry_dim, model_size, sample_layers, dim, OUTPUT, catmlp=catmlp, extra_dim=3 if use_norm else 0, channel=channel, sample_child_first=sample_child_first, dim_layer0=dim_layer0, dim_repeat_cut=dim_repeat_cut, use_symmetry_loss=use_symmetry_loss).cuda()
elif model_name == 'EncoderKdtAlign':
    has_align_feature = True
    model = EncoderKdtAlign(model_size, sample_layers, dim, OUTPUT, extra_dim=3 if use_norm else 0, num_layers=num_layers, channel=channel, sample_child_first=sample_child_first, dim_layer0=dim_layer0, dim_repeat_cut=dim_repeat_cut, use_symmetry_loss=use_symmetry_loss).cuda()
elif model_name == 'EncoderRec':
    model = EncoderRec(model_size, sample_layers, dim, OUTPUT, extra_dim=3 if use_norm else 0, num_layers=2, channel=channel, sample_child_first=sample_child_first, dim_layer0=dim_layer0, dim_repeat_cut=dim_repeat_cut, use_symmetry_loss=use_symmetry_loss).cuda()
elif model_name == 'DynEncoder':
    has_align_feature = True
    model = DynEncoder(model_size, OUTPUT=OUTPUT, extra_dim=3 if use_norm else 0, num_layer=num_layers, dim=dim, inner_dim=inner_dim, dim_layer0=dim_layer0, dim_repeat_cut=dim_repeat_cut, coo_dim=coo_dim, use_symmetry_loss=use_symmetry_loss).cuda()
elif model_name == 'SampleEncoder':
    has_align_feature = True
    encoder = lambda : Encoder(model_size, sample_layers, dim, OUTPUT, extra_dim=3 if use_norm else 0, channel=channel, sample_child_first=sample_child_first, dim_layer0=dim_layer0, dim_repeat_cut=dim_repeat_cut, use_symmetry_loss=use_symmetry_loss).cuda()
    sample_encoder = lambda : Encoder(model_size * 4, sample_layers, dim * 2, OUTPUT, extra_dim=3 if use_norm else 0, point_dim=inner_dim, channel=channel, sample_child_first=sample_child_first, dim_layer0=dim_layer0, dim_repeat_cut=dim_repeat_cut, use_symmetry_loss=use_symmetry_loss).cuda()
    # sample_encoder = Encoder(model_size, sample_layers, dim, OUTPUT, extra_dim=3 if use_norm else 0, point_dim=inner_dim, channel=channel, sample_child_first=sample_child_first, dim_layer0=dim_layer0, dim_repeat_cut=dim_repeat_cut, use_symmetry_loss=use_symmetry_loss).cuda()
    model = SampleEncoder(encoder, sample_encoder, extra_sample_coef=extra_sample_coef).cuda()
else:
    assert False, f"Unsupported {model_name}"
model.dim

In [ ]:
def hack(model):
    if upload_remove_equal:
        for i, layer in enumerate(model.layers):
            if layer.idim == layer.odim:
                layer.upload = torch.nn.Identity().cuda()
                print(f"#{i}: Set upload to identity.")
    if lowrk < dim:
        from encoder import convert_lowrk
        # pts_align = model.layers[0].pts_align
        # for i in range(6):
        #     pts_align.fc[i] = convert_lowrk(pts_align.fc[i], lowrk).cuda()
        # for l in model.layers[0].mlp.layers:
        #     l.linear = convert_lowrk(l.linear, lowrk).cuda()
        for i, layer in enumerate(model.layers):
            if layer.layer_type != 'leaf' and isinstance(layer.upload, MLP):
                for l in layer.upload.layers:
                    l.linear = convert_lowrk(l.linear, lowrk).cuda()
                print(f"#{i}: Converted to lowrk.")
    if use_small_point2embed:
        l = model.layers[0]
        l.mlp = MLP([l.idim, l.odim * 4, l.odim], init=0.25).cuda()
        print(f"Reset middle layer of point2embed")

def hack_seg(model):
    hack(model.encoder)

if isinstance(model, Encoder):
    hack(model)
elif isinstance(model, EncoderKdtAlign):
    hack(model.encoder)
    upload_remove_equal = True
    use_small_point2embed = True

    def hack_rec(model):
        if isinstance(model, Encoder):
            hack(model)
        else:
            hack(model.encoder)
            hack_rec(model.align.encoder)

    hack_rec(model.align.encoder)
elif isinstance(model, EncoderRec):
    hack(model.encoder)
elif isinstance(model, ResEncoder):
    print("feed:")
    hack(model.feed)
    # upload_remove_equal = True
    # use_small_point2embed = True
    print("layers:")
    for seg in model.layers:
        hack_seg(seg)
else:
    print("Unsupported")
    # assert False, "Unsupported"

In [ ]:
try:
    if disable_alignment:
        model.layers[0].pts_align = torch.nn.Identity()
        print("alignment disabled")
except:
    pass

In [ ]:
count = sum(p.numel() for p in model.parameters() if p.requires_grad)
count

In [ ]:
model

In [ ]:
!cat build_tree.py | grep cvg

In [ ]:
assert False, "debug part cannot be run through"

In [ ]:
model.tree.record_vec

In [ ]:
from load_modelnet import load_modelnet
k = 88
pts = h5py.File(f'./datasets/ShapeNetPart/train0.h5')['data'][k]
pts = torch.tensor(pts)
# pts, labels = load_modelnet(max_count=1, num_workers=1)
# pts = pts[0][:2048]
output = None
h5py.File(f'./datasets/ShapeNetPart/train0.h5')['label'][k]

In [ ]:
a = []
d = h5py.File(f'./datasets/ShapeNetPart/train0.h5')['label']
for i in range(2048):
    if d[i][0] == 8:
        a.append(str(i))
print(",".join(a))

In [ ]:
model.tree.use_sym

In [ ]:
def write_pts(pts, file):
    pts -= pts.mean(dim=0)
    pts /= pts.norm(dim=-1).mean()
    with open(f"{file}.txt", 'w') as file:
        file.write(f"{pts.shape[0]}\n")
        for p in pts:
            file.write("%.8lf %.8lf %.8lf\n" % (p[0].item(), p[1].item(), p[2].item()))

def read_pts(file):
    pts = []
    with open(f"{file}.txt") as file:
        for line in file:
            line = line.split()
            if len(line) == 3:
                pts.append(list(map(float, line)))
    return torch.tensor(pts)

In [ ]:
pts = transform(pts)
# pts = affine_lim(torch.pi / 4)(pts)

In [ ]:
pts, _ = augment_fn(pts)
#write_pts(pts, "/mnt/c/Users/immor/OneDrive/Academic/PointCloud/tree_builder_cpp/pts_debug")

In [ ]:
tree = model.tree
pts, output, extra = model.tree.arrange(pts, rotate=not no_prealign, extra=False, pca=True,
                                          rotate_only=rotate_only, debug=True)

In [ ]:
for i, l in enumerate(output):
    print(i, l.shape)

In [ ]:
batch_size = 32
model.cache = True
model.train()
_pts = pts.unsqueeze(0).expand(batch_size, *pts.shape)
_output = list(map(lambda x : x.unsqueeze(0).expand(batch_size, *x.shape), output))
_extra = extra.unsqueeze(0).expand(batch_size, *extra.shape)
for _ in range(8):
    model.debug = True
    feature = model(_pts, _output, _extra, perm=0)
    print(feature.shape)
    (feature.sum()).backward()

In [ ]:
output[2].shape

In [ ]:
tree.use_sym = False

In [ ]:
%matplotlib inline
def plot(points, color_layers=2, output=None, recenter=True):
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    import matplotlib.gridspec as gridspec
    import numpy as np

    points = points.clone().cpu().numpy()
    x, y, z = points[:,0], points[:,1], points[:,2]
    if recenter:
        x -= x.mean()
        y -= y.mean()
        z -= z.mean()

    color = np.array([0 for _ in range(x.shape[0])])
    
    def mark_color(z, last_ind=None):
        n = z.shape[0]
        ind = torch.tensor(z).sort()[1].numpy()
        indl, indr = ind[: n >> 1], ind[n >> 1 : ]
        if last_ind is not None:
            indl, indr = last_ind[indl], last_ind[indr]
        color[indr] += 1
        return indl, indr

    if output is not None:

        color = torch.tensor([0]).cuda()
        # print(tree.layer_size)
        if  model_name in ['ResEncoder', 'DynEncoder']:
            layers = model.feed.layers
        elif model_name in ['SampleEncoder']:
            layers = model.encoder.encoder.layers
        else:
            layers = model.layers
        for i, (layer, next_layer_size) in enumerate(zip(reversed(layers[1:]), tree.layer_size[1:])):
            next_color = torch.zeros(next_layer_size).long().cuda()
            if i < color_layers:
                color = color << 1
                right_add = 1
            else:
                right_add = 0
            assert layer.child_l.max() < next_layer_size
            assert layer.child_r.max() < next_layer_size
            next_color[layer.child_l] = color
            next_color[layer.child_r] = color + right_add
            color = next_color

        arrange = output[0]
        if len(arrange.shape) > 1:
            arrange = arrange[0]
        n = points.shape[0]
        # print(color)
        color = np.array([color[arrange == i].max().item() for i in range(n)])

    elif color_layers >= 1:
        l, r = mark_color(z)
        if color_layers >= 2:
            color *= 2
            ll, lr = mark_color(y[l], l)
            rl, rr = mark_color(y[r], r)
            if color_layers >= 3:
                color *= 2
                for p in [ll, lr, rl, rr]:
                    mark_color(x[p], p)

    colormap = np.array(['red', 'blue', 'green', 'yellow', 'grey', 'orange', 'purple', 'cyan'])
    import random
    #random.shuffle(colormap)

    fig = plt.figure(dpi=80)
    gs = gridspec.GridSpec(nrows=2, ncols=4, left=0.1, right=2.5, wspace=0.05, hspace=0.1, bottom=0.1, top=1.3)
    for i in range(8):
        ax = fig.add_subplot(gs[i // 4, i % 4], projection='3d')

        x, y, z = points[:,0], points[:,1], points[:,2]
        labx, laby, labz = 'x', 'y', 'z'
        if (i & 4) != 0:    x = -x; labx = '-x'
        if (i & 2) != 0:    y = -y; laby = '-y'
        if (i & 1) != 0:    z = -z; labz = '-z'

        lmin = min(x.min(), y.min(), z.min())
        lmax = max(x.max(), y.max(), z.max())
        ax.scatter(x, y, z, c=colormap[color], marker='.')

        ax.set_xlim(lmin, lmax)
        ax.set_ylim(lmin, lmax)
        ax.set_zlim(lmin, lmax)

        ax.set_xlabel(labx)
        ax.set_ylabel(laby)
        ax.set_zlabel(labz) 
    plt.show()

    fig = plt.figure(dpi=80)
    ncolor = color.max() + 1
    nrows = max(1, ncolor // 4)
    ncols = min(ncolor, 4)
    gs = gridspec.GridSpec(nrows=nrows, ncols=ncols, left=0.1, right=0.1 + 0.6 * ncols, wspace=0.05, hspace=0.05, bottom=0.1, top=0.1 + 0.6 * nrows)
    for i in range(ncolor):
        ax = fig.add_subplot(gs[i // 4, i % 4], projection='3d')

        p = points[color == i]
        x, y, z = p[:,0], p[:,1], p[:,2]

        lmin = min(x.min(), y.min(), z.min())
        lmax = max(x.max(), y.max(), z.max())
        ax.scatter(x, y, z, c=colormap[i], marker='.')

        ax.set_xlim(lmin, lmax)
        ax.set_ylim(lmin, lmax)
        ax.set_zlim(lmin, lmax)

        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z') 
    plt.show()

In [ ]:
output = None

In [ ]:
plot(pts.squeeze(), 3, output=output, recenter=False)

In [ ]:
plot(torch.pca_lowrank(pts)[0], 3, output=output, recenter=False)

In [ ]:
for i in range(12):
    pts = h5py.File(f'{DATASET}/train0.h5')['data'][i]
    pts = torch.tensor(pts)
    output = None
    print(i)
    tree = model.tree
    tree.use_sym = True
    pts, output, extra = model.tree.arrange(pts, rotate=not no_prealign, extra=False, pca=True,
                                          rotate_only=rotate_only, debug=False)
    plot(pts.squeeze(), 3, output=output, recenter=False)


In [ ]:
for i in range(12):
    pts = h5py.File(f'{DATASET}/train0.h5')['data'][i]
    pts = torch.tensor(pts)
    output = None
    print(i)
    tree = model.tree
    tree.use_sym = False
    pts, output, extra = model.tree.arrange(pts, rotate=not no_prealign, extra=False, pca=True,
                                          rotate_only=rotate_only, debug=False)
    plot(pts.squeeze(), 3, output=output, recenter=False)


In [ ]:
plot(read_pts("/mnt/c/Users/immor/OneDrive/Academic/PointCloud/tree_builder_cpp/pts_debug"), 3, output=output, recenter=False)

In [ ]:
model.align_dim

In [ ]:
model.load_state_dict(torch.load(f'{OUTPUT}/trained_best{prefix}.pth')['encoder'])

In [ ]:
model.extra_sample_coef = 0.1

In [ ]:
output[0]

In [ ]:
model.cached_inputs[0][0][0]

In [ ]:
plot(pts.squeeze(), 3, output=model.cached_inputs[0])
plot(pts.squeeze(), 3, output=model.cached_inputs[1])
plot(pts.squeeze(), 3, output=model.cached_inputs[2])

In [ ]:
plot_tri(pts, tri_ind)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def plot_tri(points, tri):
    points = points.numpy()
    tri = tri.numpy()

    fig = plt.figure(dpi=200)
    ax = fig.add_subplot(111, projection='3d')
    
    edges = collect_edges(tri)
    x = np.array([])
    y = np.array([])
    z = np.array([])
    for (i,j) in edges:
        x = np.append(x, [points[i, 0], points[j, 0], np.nan])      
        y = np.append(y, [points[i, 1], points[j, 1], np.nan])      
        z = np.append(z, [points[i, 2], points[j, 2], np.nan])
    ax.plot3D(x, y, z, color='g', lw='0.1')

    ax.scatter(points[:,0], points[:,1], points[:,2], color='b', s=1)


def collect_edges(tri):
    edges = set()

    def sorted_tuple(a,b):
        return (a,b) if a < b else (b,a)
    # Add edges of tetrahedron (sorted so we don't add an edge twice, even if it comes in reverse order).
    for (i0, i1, i2, i3) in tri:
        edges.add(sorted_tuple(i0,i1))
        edges.add(sorted_tuple(i0,i2))
        edges.add(sorted_tuple(i0,i3))
        edges.add(sorted_tuple(i1,i2))
        edges.add(sorted_tuple(i1,i3))
        edges.add(sorted_tuple(i2,i3))
    return edges

In [ ]:
pts = train_data[7][0]
output = None

In [ ]:
for i in range(50):
    output, arrange = tree.arrange(pts, basic=basic, debug=False, device='cpu')

In [ ]:
feature.shape

In [ ]:
for i in range(10):
    loss = model(pts.unsqueeze(0), list(map(lambda x : x.unsqueeze(0), output))).view(-1).sum()
    loss.backward()

In [ ]:
model

In [ ]:
for name, param in model.named_parameters():
    print(name, param.size())

In [ ]:
for name, param in model.named_parameters():
    if 'activate' in name:
        print(name, "\t", param.item())

In [ ]:
!cp /tmp/cppinput.txt scratch/

In [ ]:
!tail -n 5 /tmp/cppoutput.txt

# **Data**

In [ ]:
from dataset import *
import numpy as np
from load_modelnet import load_modelnet

In [ ]:
torch.multiprocessing.set_sharing_strategy('file_system')

In [ ]:
if False:
    from random import shuffle, seed
    raw_data = torch.load(f'{OUTPUT}/train_data{prefix}.pth')
    n = len(raw_data) // data_augments
    perm = list(range(n))
    seed(674433238)
    shuffle(perm)
    train_perm = perm[: -n//8]
    valid_perm = perm[-n//8: ]
    torch.save([train_perm, valid_perm], f'{OUTPUT}/train_split{prefix}.pth')

In [ ]:
make = make_data_default
if rotate_only:
    make = make_data_rotate_only
if no_prealign:
    make = make_data_no_prealign

In [ ]:
if complete_dataset:
    clouds, labels = load_modelnet(folder=DATASET, modelnet='modelnet40', split='train',  num_workers=16)
else:
    n_train = 5
    clouds = []
    labels = []
    for i in range(n_train):
        data_file = h5py.File(f'{DATASET}/train{i}.h5')
        clouds.append(torch.tensor(np.array(data_file['data'])))
        labels.append(torch.tensor(np.array(data_file['label'])))

    clouds = torch.cat(clouds, dim=0)
    labels = torch.cat(labels, dim=0)

if test_as_valid:
    train_dataset = PointCloudDataset(clouds, labels, model.tree.arrange, augment=augment, transform=transform, make=make, augment_fn=augment_fn, use_norm=use_norm)
else:
    try:
        train_perm, valid_perm = torch.load(f'{OUTPUT}/train_split{prefix}.pth')
    except:
        train_perm, valid_perm = torch.load(f'{OUTPUT}/train_split.pth')

    from math import ceil
    train_dataset = PointCloudDataset(clouds, labels, model.tree.arrange, augment=augment, transform=transform, make=make, subset=train_perm, augment_fn=augment_fn, use_norm=use_norm) #, extra_labels=clouds)
    valid_dataset = PointCloudDataset(clouds, labels, model.tree.arrange, augment=1, transform=transform, make=make, subset=valid_perm, use_norm=use_norm)

In [ ]:
if complete_dataset:
    clouds, labels = load_modelnet(folder=DATASET, modelnet='modelnet40', split='test', num_workers=16)
else:
    n_test = 2
    clouds = []
    labels = []
    for i in range(n_test):
        data_file = h5py.File(f'{DATASET}/test{i}.h5')
        clouds.append(torch.tensor(np.array(data_file['data'])))
        labels.append(torch.tensor(np.array(data_file['label'])))

    clouds = torch.cat(clouds, dim=0)
    labels = torch.cat(labels, dim=0)

test_dataset = PointCloudDataset(clouds, labels, model.tree.arrange, augment=1, make=make, transform=transform, use_norm=use_norm)

if test_as_valid:
    valid_dataset = test_dataset

In [ ]:
len(train_dataset), len(valid_dataset), len(test_dataset)

In [ ]:
target = [] # ['valid', 'test'], ['train', 'valid', 'test']

import gc
gc.collect()

for name, dataset in zip(['train', 'valid', 'test'], [train_dataset, valid_dataset, test_dataset]):
    from tqdm import tqdm

    if name not in target:
        continue

    num_workers = 16
    batch_size = 8

    data_init = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, 
                                            num_workers=num_workers, collate_fn=placeholder, pin_memory=False, drop_last=False)
    logging.info(f"Init {name}")

    counter = 0
    mem = []

    for i, data in enumerate(tqdm(data_init)):
        mem += data
        # logging.debug(f"Init {name}: {i+1}/{len(data_init)}")

        if len(mem) >= 2048:
            torch.save(mem, f'{OUTPUT}/{name}_data{prefix}.{counter}.pth')
            counter += 1
            mem.clear()
            gc.collect()
            
    torch.save(mem, f'{OUTPUT}/{name}_data{prefix}.{counter}.pth')
    del mem
    gc.collect()


# **Train**

## Defs

In [ ]:
torch.multiprocessing.set_sharing_strategy('file_descriptor')

In [ ]:
from dataset import make_batch_train, make_batch_eval

global batch_size
num_workers = 16
batch_size = 128

for name, dataset in zip(['train', 'valid', 'test'], [train_dataset, valid_dataset, test_dataset]):
    print(f"Loading {name}")
    dataset.mem = None

    try:
        tmp = torch.load(f'{OUTPUT}/{name}_data{prefix}.pth')
        dataset.mem = tmp
        assert len(dataset.mem) == len(dataset), f'Size unmatch: {len(dataset.mem)} != {len(dataset)}'
        num_workers = 1
        continue
    except:
        print("Try part mode")

    dataset.mem = []
    for i in range(0, 1000000000):
        try:
            tmp = torch.load(f'{OUTPUT}/{name}_data{prefix}.{i}.pth')
            dataset.mem += tmp
            print(f"Loaded part {i} # = {len(tmp)}")
        except:
            break
    if len(dataset.mem) > 0:
        num_workers = 1
        continue

    print("Use force online")
    dataset.mem = None
    dataset.force_online = True

    
mbtrain = make_batch_generator(pca_augment=pca_augment and not no_prealign) #make_batch_train


def inf_iter(a):
    while True:
        for k in a:
            yield(k)

num_workers

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=mbtrain, pin_memory=True, drop_last=True, prefetch_factor=8 if num_workers>1 else 2)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=make_batch_eval, pin_memory=True, drop_last=False, prefetch_factor=8 if num_workers>1 else 2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=make_batch_eval, pin_memory=True, drop_last=False, prefetch_factor=8 if num_workers>1 else 2)


In [ ]:
torch.manual_seed(674433238)

class Classifier(torch.nn.Module):
    def __init__(self, idim, odim, dropout=0):
        super(Classifier, self).__init__()
        self.relu = torch.nn.ReLU()

        self.linear1 = make_lowrk(idim, min(idim // 2, 1024), lowrk)
        self.bn1 = torch.nn.BatchNorm1d(min(idim // 2, 1024))

        # self.linear2 = make_lowrk(idim // 2, idim // 4, lowrk)
        # self.bn2 = torch.nn.BatchNorm1d(idim // 4)
        self.linear2 = torch.nn.Identity()
        self.bn2 = torch.nn.Identity()

        self.dropout = torch.nn.Dropout(dropout)

        # self.linear3 = make_lowrk(idim // 4, odim, lowrk)
        self.linear3 = make_lowrk(min(idim // 2, 1024), odim, lowrk)

    def forward(self, ans):
        ans = self.linear1(ans)
        ans = self.bn1(ans)
        ans = self.relu(ans)
        # ans = self.dropout(ans)

        ans = self.linear2(ans)
        ans = self.bn2(ans)
        ans = self.relu(ans)
        ans = self.dropout(ans)
        
        ans = self.linear3(ans)
        return ans

if 'orig' in prefix:
    from encoder import make_lowrk
    linear = Classifier(model.dim, num_classes, 1 - 1/2).cuda()

    align_num = 1 if has_align_feature else 0
    try:
        align_num = model.align_num
    except:
        pass
    linear_align = torch.nn.ModuleList([Classifier(model.align_dim, num_classes, 1 - 1/2).cuda() for _ in range(align_num)])
    # linear = MLP([model.dim, min(1024, model.dim // 2), min(512, model.dim // 4), num_classes], last_bn=False).cuda()
    # linear_align = MLP([model.align_dim, min(1024, model.align_dim // 2), min(512, model.align_dim // 4), num_classes], last_bn=False).cuda() if has_align_feature else torch.nn.Identity().cuda()
else:
    align_num = 1 if has_align_feature else 0
    try:
        align_num = model.align_num
    except:
        pass
    linear = MLP([model.dim, min(1024, model.dim // 2), min(512, model.dim // 4), num_classes], last_bn=False).cuda()
    linear_align = torch.nn.ModuleList([MLP([model.align_dim, min(1024, model.align_dim // 2), min(512, model.align_dim // 4), num_classes], last_bn=False).cuda() for _ in range(align_num)]) #if has_align_feature else torch.nn.Identity().cuda()
    # linear = Classifier(model.dim, num_classes, 1 - 1/4).cuda()
    # linear_align =  Classifier(model.dim, num_classes, 1 - 1/4).cuda()

print(linear)
print(linear_align)

In [ ]:
def evaluate(model, linear, loader, noprint=False, perms=[None]):
    correct = 0
    total = 0

    if not noprint:
        logging.info(f"loader # = {len(loader)}")

    print_epoch = 1

    model.eval()
    linear.eval()
    activate = lambda x : x

    for epoch, (input, label) in enumerate(loader):
        with torch.no_grad():
            label = label.squeeze(-1).cuda()
            for perm in perms:
                result = activate(linear(model(*input, perm=perm).contiguous().view(label.size(0), -1)))

                correct += (result.argmax(dim=-1) == label.cuda()).sum().item()
                total += label.size()[0]

        if not noprint:
            if (epoch // batch_size + 1) % print_epoch == 0:
                logging.debug(f"test #{epoch} correct = {'%.6lf' % (correct / total)}")

    if not noprint:
        logging.info(f"Done: score = {'%.8lf' % (correct / total)}")

    model.train()
    linear.train()  
    
    return correct / total

In [ ]:
from random import choice, randint
import build_tree
import torch

def get_trans(n=3):
    if n == 0:  return [None]
    return [randint(0, len(build_tree.transforms) - 1) for _ in range(n)]


In [ ]:
# if 'orig' in prefix:
#     ckpt = torch.load(f"{OUTPUT}/pretrained_affine_pca_nosample_x16_20211207.pth")
#     model.load_state_dict(ckpt['encoder'])
#     linear.load_state_dict(ckpt['linear'])
#     print("Pretrain loaded")

In [ ]:
global current_epoch
global best_vres
current_epoch = 0
best_vres = -1.0

def train(epoch=100000, lr=1e-4, valid_result_threshold=1.0, mode=1, ckpt=None):
    global current_epoch
    global best_vres
    global batch_size

    logging.info(f"train epoch = {current_epoch + 1} ~ {epoch} threshold = {valid_result_threshold}")

    model.train()
    linear.train()
    activate = lambda x : x

    cum_loss = 0
    cum_loss_align = 0

    batch_scale = 1 # 1
    epoch_scale = 8 # 8
    num_trans = 1 if permute_augment else 0


    print_epoch = 20
    valid_epoch = 20
    epoch_since = 0
    save_epoch = 100
    cut_epoch = 10000000000
    correct = 0
    total = 0

    temperature = 1
    threshold = -1.0

    crit = torch.nn.CrossEntropyLoss()
    crit_align = torch.nn.CrossEntropyLoss()
    opt = torch.optim.Adam(list(model.parameters()) + list(linear.parameters()) + list(linear_align.parameters()), lr=lr)

    if ckpt is not None:
        model.load_state_dict(ckpt['encoder'])
        linear.load_state_dict(ckpt['linear'])
        linear_align.load_state_dict(ckpt['linear_align'])
        opt.load_state_dict(ckpt['opt'])
    
    warmup = 0
    if mode == 1:
        sch = torch.optim.lr_scheduler.ExponentialLR(opt, 0.998)
    elif mode == 2:
        opt.param_groups[0]['lr'] = lr
        opt.param_groups[0]['initial_lr'] = lr
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(opt, T_0=10000 // valid_epoch, T_mult=2, eta_min=lr / 100)
    elif mode == 3:
        lr /= 5
        opt.param_groups[0]['lr'] = lr
        opt.param_groups[0]['initial_lr'] = lr
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(opt, T_0=10000 // valid_epoch, T_mult=2, eta_min=lr / 100)
    else:
        assert False


    def save(epoch):
        torch.save({
            'encoder': model.state_dict(),
            'linear': linear.state_dict(),
            'linear_align': linear_align.state_dict(),
            'opt': opt.state_dict(),
            'sch': sch.state_dict(),
            'best_vres': best_vres,
        }, f"{OUTPUT}/trained_{epoch}.pth")

    if not test_as_valid:
        save(current_epoch)

    train_iter = inf_iter(train_loader)

    for epoch in range(current_epoch + 1, current_epoch + epoch + 1):

        current_epoch = epoch

        align_coef = 0
        align_coef_sum = 1
        for _ in range(epoch_scale):
            loss = torch.tensor(0.).cuda()
            loss_align = torch.tensor(0.).cuda()
            for _ in range(batch_scale):
                input, label = next(train_iter)
                label = label.squeeze(-1).cuda()
                for iperm in get_trans(num_trans):
                    
                    features = model(*input, perm=iperm).reshape(batch_size, -1)
                    logits = linear(features)
                    loss += crit(logits / temperature, label.cuda())
                    if has_align_feature:
                        align_coef = 1
                        align_coef_sum = 0
                        if not isinstance(model.align_feature, list):
                            model.align_feature = [model.align_feature]

                        for fea, cls in zip(model.align_feature, linear_align):
                            align_coef_sum += align_coef
                            logits_align = cls(fea)
                            loss_align += align_coef * crit_align(logits_align / temperature, label.cuda())
                            align_coef *= 0.5

                    with torch.no_grad():
                        correct += (logits.argmax(dim=-1) == label.cuda()).sum().item()
                        total += label.size()[0]
                        epoch_since += 1
            
            # assert loss.isnan().sum() == 0
            cum_loss += loss.item()
            cum_loss_align += loss_align.item()
            opt.zero_grad()
            # loss.backward()
            if 'orig' in prefix:
                # (loss / loss.clamp(min=1e-6).item()).backward()
                all = (loss / loss.clamp(min=1e-6).item()) + (loss_align / loss_align.clamp(min=1e-6).item())
                all.backward()
            else:
                all = loss + loss_align
                all.backward()
            opt.step()

        if cum_loss / epoch_since < threshold:
            epoch_scale, batch_scale = batch_scale, epoch_scale
            
            logging.info("Threshold Reached")
            threshold = -1e10
            
        if epoch <= 5 or epoch % print_epoch == 0:
            valid_str = ""
            func = logging.debug

            stop_training = False
            if epoch % valid_epoch == 0:
                vres = evaluate(model, linear, valid_loader, noprint=True)
                valid_str = f"valid = {'%.6lf' % vres}"
                stop_training = (vres >= valid_result_threshold)
                if vres > best_vres:
                    best_vres = vres
                    torch.save({
                        'encoder': model.state_dict(),
                        'linear': linear.state_dict()
                    }, f"{OUTPUT}/trained_best{prefix}.pth")
                    valid_str += " updated"

                func = logging.info

            loss_sch = (cum_loss + cum_loss_align) / epoch_since
            func(f"train #{epoch} lr = {'%.2e' % opt.param_groups[0]['lr']} loss = {'%.6lf / %.6lf' % (cum_loss / epoch_since, cum_loss_align / epoch_since / align_coef_sum)} train = {'%.6lf' % (correct / total)} {valid_str}")

            # step here to make output beautiful
            if epoch > warmup and epoch % print_epoch == 0:
                sch.step()

            epoch_since = cum_loss = cum_loss_align = correct = total = 0

            if stop_training:
                break

        

        if epoch % save_epoch == 0:
            if not test_as_valid:
                save(epoch)
            tres = best_vres if test_as_valid else evaluate(model, linear, test_loader, noprint=True)
            logging.info(f"Saved{'(Skipped)' if test_as_valid else ''} test = {'%.6lf' % tres}")

        if epoch % cut_epoch == 0:
            if batch_size > 8:
                batch_size //= 2
                epoch_scale *= 2

            logging.info(f"Cut batch_size = {batch_size} epoch_scale = {epoch_scale}")

## Loop

In [ ]:
train()